In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications import ResNet50, ResNet152, InceptionV3, InceptionResNetV2, Xception, DenseNet121, DenseNet169, DenseNet201
from tensorflow.keras.applications.resnet import preprocess_input as resnet_preprocess_input
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess_input
from tensorflow.keras.applications.xception import preprocess_input as xception_preprocess_input
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess_input
from tensorflow.keras.callbacks import LearningRateScheduler
import math

# Set random seed for reproducibility
SEED = 42
tf.random.set_seed(SEED)

# Check GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Directories for training and testing data
train_dir = "/kaggle/input/skin-lesion-input-dataset-123/Skin Lesion Input Dataset/Train_Set_Images"
test_dir = "/kaggle/input/skin-lesion-input-dataset-123/Skin Lesion Input Dataset/Test_Set_Images"

IMG_SIZE = (224, 224)
BATCH_SIZE = 128

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Function to build models dynamically
def build_model(model_name):
    if model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        preprocess_func = resnet_preprocess_input
    elif model_name == 'ResNet152':
        base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        preprocess_func = resnet_preprocess_input
    elif model_name == 'InceptionV3':
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        preprocess_func = inception_preprocess_input
    elif model_name == 'InceptionResNetV2':
        base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        preprocess_func = inception_preprocess_input
    elif model_name == 'Xception':
        base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        preprocess_func = xception_preprocess_input
    elif model_name == 'DenseNet121':
        base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        preprocess_func = densenet_preprocess_input
    elif model_name == 'DenseNet169':
        base_model = DenseNet169(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        preprocess_func = densenet_preprocess_input
    elif model_name == 'DenseNet201':
        base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        preprocess_func = densenet_preprocess_input

    
    # Build the model
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dropout(0.5),  # Set dropout rate to 0.5
        Dense(train_generator.num_classes, activation='softmax')
    ])
    return model

# List of models to train
models_to_train = ['InceptionResNetV2']

# Training loop
for model_name in models_to_train:
    print(f"\nTraining {model_name}...\n")
    model = build_model(model_name)

    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Learning Rate Scheduler
    def step_decay(epoch):
        initial_lr = 0.001
        drop_rate = 0.1
        epochs_drop = 5
        new_lr = initial_lr * math.pow(drop_rate, math.floor((1 + epoch) / epochs_drop))
        return new_lr
    
    lr_scheduler = LearningRateScheduler(step_decay)
    
    # Train the model
    history = model.fit(
        train_generator,
        epochs=20,
        validation_data=test_generator,
        callbacks=[lr_scheduler],
        verbose=1
    )

    # Evaluate the model
    train_loss, train_acc = model.evaluate(train_generator)
    test_loss, test_acc = model.evaluate(test_generator)

    print(f"{model_name} Training Accuracy: {train_acc*100:.2f}%")
    print(f"{model_name} Testing Accuracy: {test_acc*100:.2f}%")

Num GPUs Available:  2
Found 5823 images belonging to 7 classes.
Found 1459 images belonging to 7 classes.

Training InceptionResNetV2...

219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1733405544.151716     102 service.cc:145] XLA service 0x7ae270008a80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733405544.151770     102 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1733405544.151774     102 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1733405675.809558     102 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_104', 72 bytes spill stores, 72 bytes spill loads
ptxas warning : Registers 

 3/46 ━━━━━━━━━━━━━━━━━━━━ 1:25 2s/step - accuracy: 0.2174 - loss: 1.9419

I0000 00:00:1733405786.664688     102 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_105', 72 bytes spill stores, 72 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_104', 128 bytes spill stores, 128 bytes spill loads



46/46 ━━━━━━━━━━━━━━━━━━━━ 454s 5s/step - accuracy: 0.5084 - loss: 1.3163 - val_accuracy: 0.1693 - val_loss: 3.8525 - learning_rate: 0.0010
Epoch 2/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 107s 2s/step - accuracy: 0.7778 - loss: 0.5958 - val_accuracy: 0.1145 - val_loss: 13.9797 - learning_rate: 0.0010
Epoch 3/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 106s 2s/step - accuracy: 0.8548 - loss: 0.3920 - val_accuracy: 0.3770 - val_loss: 2.5425 - learning_rate: 0.0010
Epoch 4/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 106s 2s/step - accuracy: 0.8893 - loss: 0.3103 - val_accuracy: 0.5374 - val_loss: 1.9571 - learning_rate: 0.0010
Epoch 5/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 106s 2s/step - accuracy: 0.9316 - loss: 0.1922 - val_accuracy: 0.6408 - val_loss: 1.4728 - learning_rate: 1.0000e-04
Epoch 6/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 106s 2s/step - accuracy: 0.9687 - loss: 0.0927 - val_accuracy: 0.6820 - val_loss: 1.3215 - learning_rate: 1.0000e-04
Epoch 7/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 106s 2s/step - accuracy: 0.9834 - loss: 0.0537 - val_accur